In [49]:
import time
import sys
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile
import numpy as np

cv2.__version__

'4.11.0'

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


descompacta tudo

In [51]:
caminhoPose = '/content/drive/MyDrive/projetinhos/libras/pose.zip'
objZipado = zipfile.ZipFile(file = caminhoPose, mode='r')
objZipado.extractall('./')

In [52]:
caminhoImg = '/content/drive/MyDrive/projetinhos/libras/imagens.zip'
objZipado = zipfile.ZipFile(file = caminhoImg, mode='r')
objZipado.extractall('./')

In [53]:
caminhoMod = '/content/drive/MyDrive/projetinhos/libras/modulos.zip'
objZipado = zipfile.ZipFile(file = caminhoMod, mode='r')
objZipado.extractall('./')

fazer "criacao de lib" os codigo abaixo foi feito pelo tiether para verificar movimentos. e para fazer isso, conect som o sitema e faco a "importacao"

In [54]:
sys.path.append('/content/modulos')#transforma em variaavel de ambiente
sys.path

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/modulos',
 '/content/modulos',
 '/content/modulos',
 '/content/modulos',
 '/content/modulos']

In [55]:
import extrator_CORPO

In [56]:
arqProto = '/content/pose/body/mpi/pose_deploy_linevec_faster_4_stages.prototxt'
arqPesos = '/content/pose/body/mpi/pose_iter_160000.caffemodel'

In [57]:
numPontos = 15
paresPontos = [[0, 1], [1, 2], [2, 3], [3, 4], [1, 5], [5, 6], [6, 7], [1, 14],
                [14, 8], [8, 9], [9, 10], [14, 11], [11, 12], [12, 13]]

In [58]:
corPontoA, corPontoB, corLinha = (128,85,255), (128,255,255), (128,255,119)
corTxtPonto, corTxtInicioMovim, corTxtFzdMovim = (0,0,0), (234,169,255), (234,255,127)

tamFont, tamLinha, tamCircle, espessuma = 0.8, 2, 8, 5
fonte = cv2.FONT_HERSHEY_SIMPLEX

In [59]:
entradaW, entradaH = 256, 256

In [60]:
validPernaJunta, validPernaLonge = 0,0
validBracoJunta, validBracoLonge = 0,0

In [61]:
video = '/content/imagens/body/videos/original_3.mp4'
captura = cv2.VideoCapture(video)
conectado, frame = captura.read() #1 parametro:verifica se o video foi devidamete vinvulado, 2 parametro:captura cada frame do video, daria para fazer o tradutor partindo de video ao vivo (tipo discurso eleitoral)

In [62]:
result = './polichinelo.avi'
gravarVideo = cv2.VideoWriter(result, cv2.VideoWriter_fourcc(*'XVID'), 10, (frame.shape[1], frame.shape[0]))

In [63]:
modeloTreinamento = cv2.dnn.readNetFromCaffe(arqProto, arqPesos)

In [66]:
limite = 0.1

#tem que processar enquanto houver video
while(cv2.waitKey(1)<0):
  tempo = time.time()
  conectado, frame = captura.read()
  videoCopia = np.copy(video)

  if not conectado:
    cv2.waitKey()
    break

  videoW = video.shape[1]
  videoH = video.shape[0]

  #fazendo mascara
  mapaSuave = cv2.resize(frame, (entradaW, entradaH))
  mapaSuave = cv2.cvtColor(mapaSuave, cv2.COLOR_BGR2GRAY)
  fundo = np.uint8(mapaSuave > limite)

  #conversao do tipo da img
  blobEntrada = cv2.dnn.blobFromImage(video, 1.0 / 255,
                                      (entradaW, entradaH),
                                      (0, 0, 0), swapRB=True, crop=False)
  modelo.setInput(blobEntrada)
  saida = modelo.forward()

  altura, largura,pontos  = saida.shape[2], saida.shape[3], []